<a href="https://colab.research.google.com/github/ravitejaadapa12/python-assignment/blob/main/prompt%20engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install necessary libraries
!pip install gradio -q
!pip install groq -q
!pip install pandas -q

import os
import json
import gradio as gr
from groq import Groq
from google.colab import userdata
from google.colab import drive

# API Key Setup
groq_api_key = userdata.get('groqAPI')  # Retrieves the secret
os.environ['GROQ_API_KEY'] = groq_api_key  # Assigns the retrieved secret to the environment variable
drive.mount('/content/drive')


def extract_summary_and_authors(article_text):
    """Extract summaries for different audiences and author names from the article."""
    try:
        # Ensure API key is set
        if 'GROQ_API_KEY' not in os.environ:
            raise ValueError("Groq API key not found in environment variables")

        client = Groq(api_key=os.environ['GROQ_API_KEY'])

        # Prompt for Summarization with Few-Shot Examples
        prompt = f"""
You are an advanced summarization and information extraction assistant. Your task is to generate summaries for three audiences and extract author names.

ARTICLE:
{article_text}

TASKS:
1. Provide summaries tailored for:
   - An Expert in the field (include advanced terminology and detailed explanations)
   - A High School Student (simple explanations with relevant examples)
   - A General Adult Reader (easy-to-understand summary without technical jargon)
2. Ensure each summary maintains accuracy and focus.
3. Extract all author names.

EXAMPLES:
ARTICLE: "Quantum Mechanics is a fundamental branch of physics..."
- Expert: "Quantum Mechanics explores particle-wave duality using Schrödinger's equations..."
- High School Student: "Quantum Mechanics explains how tiny particles can behave like waves..."
- General Adult: "Quantum Mechanics studies how particles behave differently at microscopic levels."
- Authors: ["Albert Einstein", "Niels Bohr"]

OUTPUT FORMAT (STRICT JSON):
{{
    "summaries": {{
        "expert": "",
        "high_school_student": "",
        "general_adult": ""
    }},
    "authors": []
}}
"""

        response = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a structured summarization and information extraction assistant. Output ONLY the requested JSON."
                },
                {"role": "user", "content": prompt}
            ],
            model="mixtral-8x7b-32768",
            temperature=0.1,
            top_p=0.9,
            max_tokens=2000,
            response_format={"type": "json_object"}
        )

        # Parse response
        response_content = response.choices[0].message.content.strip()
        data = json.loads(response_content)

        # Validate JSON structure
        if not isinstance(data, dict) or 'summaries' not in data or 'authors' not in data:
            return {"error": "Invalid JSON response structure"}

        return data

    except Exception as e:
        print(f"Error during extraction: {e}")
        import traceback
        traceback.print_exc()
        return {"error": str(e)}


def save_to_txt(data, output_path):
    """Save summaries and authors to a TXT file."""
    try:
        with open(output_path, 'w') as file:
            file.write("🔍 **Summaries and Authors** 🔍\n\n")
            file.write("🧠 **Expert Audience:**\n")
            file.write(data['summaries'].get('expert', '') + "\n\n")
            file.write("🎓 **High School Student Audience:**\n")
            file.write(data['summaries'].get('high_school_student', '') + "\n\n")
            file.write("👥 **General Adult Audience:**\n")
            file.write(data['summaries'].get('general_adult', '') + "\n\n")
            file.write("✍️ **Authors:**\n")
            file.write(", ".join(data.get('authors', [])) + "\n")
        return f"✅ Summaries and authors saved to {output_path}"
    except Exception as e:
        print(f"Error saving to TXT: {e}")
        return {"error": str(e)}


def process_files(article_path):
    """Process the article file and generate summaries."""
    try:
        # Read the article
        with open(article_path, 'r') as file:
            article_text = file.read()

        # Extract summaries and authors
        extracted_info = extract_summary_and_authors(article_text)

        if "error" in extracted_info:
            return json.dumps(extracted_info, indent=2)

        # Save to TXT
        output_path = '/content/summaries_and_authors.txt'
        save_status = save_to_txt(extracted_info, output_path)

        return save_status

    except Exception as e:
        print(f"Unexpected error: {e}")
        import traceback
        traceback.print_exc()
        return json.dumps({"error": str(e)})


def main():
    # Gradio Interface
    iface = gr.Interface(
        fn=process_files,
        inputs=[
            gr.File(label="Upload Article (TXT)")
        ],
        outputs=gr.Textbox(label="Processing Status"),
        title="📚 Advanced Text Summarizer with Author Extraction",
        description="Summarizes an article for experts, high school students, and general readers. Also extracts author names."
    )

    iface.launch()


if __name__ == "__main__":
    # Ensure Groq API key is set
    if not os.environ.get("GROQ_API_KEY"):
        print("❌ Please set your GROQ_API_KEY environment variable.")
    else:
        main()


Mounted at /content/drive
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7a84de4e564ac44324.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
